# The Fashion MNIST Dataset

### Loading the data

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from PIL import Image
from helpers import display_dataframe # This is a custom function I wrote to display the images within the pandas dataframe.

In [2]:
# Loading the data
(Xtr, ytr), (Xte, yte) = tf.keras.datasets.fashion_mnist.load_data()

In [3]:
label_dict = {
    0:'Top',        1:'Trouser',
    2:'Pullover',   3:'Dress',
    4:'Coat',       5:'Sandal',
    6:'Shirt',      7:'Sneaker',
    8:'Bag',        9:'Boot'}

def generate_dataframe(X, y, label_dict=label_dict):
    data = pd.DataFrame([{'image':Image.fromarray(xi), 'array':xi.ravel(), 'target':yi} for xi, yi in zip(X,y)])
    data['target'] = data['target'].map(label_dict)
    return data

data = generate_dataframe(Xtr, ytr)

In [4]:
data.head()

,array,image,target
0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",<PIL.Image.Image image mode=L size=28x28 at 0x...,Boot
1,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 41, 188, 103, 5...",<PIL.Image.Image image mode=L size=28x28 at 0x...,Top
2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 22, 118, 24, 0, 0,...",<PIL.Image.Image image mode=L size=28x28 at 0x...,Top
3,"[0, 0, 0, 0, 0, 0, 0, 0, 33, 96, 175, 156, 64,...",<PIL.Image.Image image mode=L size=28x28 at 0x...,Dress
4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",<PIL.Image.Image image mode=L size=28x28 at 0x...,Top


In [5]:
# The helper function display_da
display_dataframe(data.head())

,array,image,target
0,"A numpy array of shape (784,) and dtype uint8","<img src=""data:image/jpeg; base64, /9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBwcJCQgKDBQNDAsLDBkSEw8UHRofHh0aHBwgJC4nICIsIxwcKDcpLDAxNDQ0Hyc5PTgyPC4zNDL/wAALCAAcABwBAREA/8QAHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL/8QAtRAAAgEDAwIEAwUFBAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2JyggkKFhcYGRolJicoKSo0NTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1dnd4eXqDhIWGh4iJipKTlJWWl5iZmqKjpKWmp6ipqrKztLW2t7i5usLDxMXGx8jJytLT1NXW19jZ2uHi4+Tl5ufo6erx8vP09fb3+Pn6/9oACAEBAAA/APn+tbw1oNx4m8QWmkWx2yXD4LkZCADJJ+gFbviL4a63oc7COE3MW4hdn38duD976jNc9daDqllIsc9lKrMu4YGeMkdR7gj8KzcV7H8BtEvV16+1iWCeG1Wz8mOV02pIzupwCeuAp6Z98cZ90aIzLIlw0c0ZJ4KgjHoeOa+evjS9n/wnMcNxBPCYLKONFhA2FNzMpGenDcgd816V4K03wefC+m3NlpVhP+5QSXBiR5fMx825iMg5zwce3FdbOzTwgW90lu6uCm8eYrL02soIyCPQgggEdMGQ3cluiPNK0rJwrRQBNueuMkt+teNfGKxsdY8WWdxNqcNo66eieXMwVsb5DnH415Hp2rajpE5n02/urOUjBe3laMkehIPIrVm8eeLrhNknibVivoLtx/I1UPinxC3XXtUP1vJP8ay5JZJpGkldnduSzHJP41//2Q=="" alt=""Black box"">",Boot
1,"A numpy array of shape (784,) and dtype uint8","<img src=""data:image/jpeg; base64, /9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBwcJCQgKDBQNDAsLDBkSEw8UHRofHh0aHBwgJC4nICIsIxwcKDcpLDAxNDQ0Hyc5PTgyPC4zNDL/wAALCAAcABwBAREA/8QAHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL/8QAtRAAAgEDAwIEAwUFBAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2JyggkKFhcYGRolJicoKSo0NTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1dnd4eXqDhIWGh4iJipKTlJWWl5iZmqKjpKWmp6ipqrKztLW2t7i5usLDxMXGx8jJytLT1NXW19jZ2uHi4+Tl5ufo6erx8vP09fb3+Pn6/9oACAEBAAA/AOB+E3gyz8V6xez6lE01jYRqWiDFQ8jk7QSOcYVjxjoK9hlufDultLpcPhSa5itZdjra6NFNCshiEh5Yj5tmCTTjpvhnxg9zYX/huSCe3SMyLeWC20yI+7aUZTnb8jcA446V81a7pcmh69qGlytue0uHh3f3gpIB/Ec/jXSeBfiJf+CvMt7ezsp7a4lV5vORg/HHDKR2z1z1NfQF94U8Oapez3l7o9vczzkGSSR5MvjAGcMB0C9B2rC8Uz2Xw28LXGp+HdIsoJ5JIoCr+YyNyx5y2eMtjBHJr5517WrrxFrdzq16sS3FwwLiJdqjAAGB9AKzq+zkXjAAGcnivP8A4zLu8ATf7N1C/wCrD+tfONFfUf8AwlV5AmBbWrbQACwf0Ho1cV8S/Edzqfg66t5be2RTLFzGHzw2e7GvEa//2Q=="" alt=""Black box"">",Top
2,"A numpy array of shape (784,) and dtype uint8","<img src=""data:image/jpeg; base64, /9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBwcJCQgKDBQNDAsLDBkSEw8UHRofHh0aHBwgJC4nICIsIxwcKDcpLDAxNDQ0Hyc5PTgyPC4zNDL/wAALCAAcABwBAREA/8QAHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL/8QAtRAAAgEDAwIEAwUFBAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2JyggkKFhcYGRolJicoKSo0NTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1dnd4eXqDhIWGh4iJipKTlJWWl5iZmqKjpKWmp6ipqrKztLW2t7i5usLDxMXGx8jJytLT1NXW19jZ2uHi4+Tl5ufo6erx8vP09fb3+Pn6/9oACAEBAAA/APn+vX/DGqaVa6HZw6r4X07VQIFMLyxqrpkZIJ2/MM5P4nrXH+PZUuLu0njs7a0VlcLDbRBERQRhRgc49fc1yFFenaeANMsx6QRj/wAdFc/41H7uwOOcyc+3yf8A165GivUbVSlnAh6rEgP/AHyKwPGa7rK0fH3JGHX1A/wrjaK6zRPEN5Ni2mWKQImA7Kdxx6kGsfWdXudSmEc2xY4idqRjAz6nPJNZdf/Z"" alt=""Black box"">",Top
3,"A numpy array of shape (784,) and dtype uint8","<img src=""data:image/jpeg; base64, /9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBwcJCQgKDBQNDAsLDBkSEw8UHRofHh0aHBwgJC4nICIsIxwcKDcpLDAxNDQ0Hyc5PTgyPC4zNDL/wAALCAAcABwBAREA/8QAHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL/8QAtRAAAgEDAwIEAwUFBAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2JyggkKFhcYGRolJicoKSo0NTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1dnd4eXqDhIWGh4iJipKTlJWWl5iZmqKjpKWmp6ipqrKztLW2t7i5usLDxMXGx8jJytLT1NXW19jZ2uHi4+Tl5ufo6erx8vP09fb3+Pn6/9oACAEBAAA/APHfBp0z/hKLRdWSNrd22KZseWrnhS4PG3PXPHc8A17c95bWoiiawsW8n/VK9oj+Uf8AZ4+X8K434kXmnXOjGe9t4n1CZgttKiBZMjqS2MlQD0Oeo6da8lor3OZzeQWsqFGZ7dCwEijB2D34rg/iIXM2nbiCPLfowIByCeh9MVxFaGh2iX+uWVtIu6N5V3r6qOSPyBr1eS1iklWZoIWdoppM+WOQFBB6fWs7xzpUUnhiSWKFFNrLn5VA29PTsVZPyFeU1c0vUZdJ1GK9gSN5It20SAleVI7EetdHJ441P7NbuILMGOLygAjYKkMv970qvqPjrVdTsLi0mitEW4wJHjjIY4VV9cdFHauYr//Z"" alt=""Black box"">",Dress
4,"A numpy array of shape (784,) and dtype uint8","<img src=""data:image/jpeg; base64, /9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBwcJCQgKDBQNDAsLDBkSEw8UHRofHh0aHBwgJC4nICIsIxwcKDcpLDAxNDQ0Hyc5PTgyPC4zNDL/wAALCAAcABwBAREA/8QAHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL/8QAtRAAAgEDAwIEAwUFBAQAAAF9AQIDAAQRBRIhMUEGE1

In [6]:
data.shape

(60000, 3)

# Generating a "Simple" version of this dataset

In [7]:
from sklearn.model_selection import train_test_split

# 60,000 are a few too many images for most computers, let's shring that down a bit
# The goal is to train on 4,000 images and test on 1,000

data_small = data.sample(5000, random_state=1).reset_index(drop=True)

X = pd.DataFrame({i:x.ravel() for i,x in enumerate(data_small['array'])}).T
y = data_small['target']

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=1)

# Finding the baseline

In [8]:
# First (and pretty much foremost) find the baseline:
y_test.value_counts(normalize=True)

Trouser     0.110
Shirt       0.109
Sandal      0.108
Sneaker     0.106
Bag         0.102
Dress       0.101
Coat        0.097
Pullover    0.094
Boot        0.092
Top         0.081
Name: target, dtype: float64

### The baseline is 11%
Any accuracy below 11% is worse than random, but even a one-in-five correct prediction represents a 100% improvement over random.

# Basic Model

- This model simply uses a 3-layer decision tree to predict on the images


In [9]:
import warnings
warnings.filterwarnings('ignore')
    
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline


model = Pipeline([('dtc', DecisionTreeClassifier(max_depth=3))])

In [10]:
model.fit(X_train, y_train)
model.score(X_test, y_test)

0.477

# Not so basic Model

- This model fist scales the data so that each of the 784 pixels' brightness has mean 0 and standard deviation 1
- It then reduces the 784 dimensional data to 20 dimensions by using linear PCA
- Lastly it implements a 3-layer decision tree

In [11]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

model = Pipeline([('scaler', StandardScaler()),
                  ('pca', PCA(n_components=20)),
                  ('dtc', DecisionTreeClassifier(max_depth=3))])

In [12]:
model.fit(X_train, y_train)
model.score(X_test, y_test)

0.483

# A Non-basic Model

- This model fist scales the data so that each of the 784 pixels' brightness has mean 0 and standard deviation 1
- It then reduces the 784 dimensional data to 20 dimensions by using linear PCA
- It then expands these 20 features into 8,000 (i.e. $20^3$) by considering all polynomial combinations (degree=3)
- It then reduces the 8,000 dimensional to 20 dimensions by using linear PCA
- Lastly it implements a 3-layer decision tree

In [13]:
from sklearn.preprocessing import PolynomialFeatures

model = Pipeline([('scaler', StandardScaler()),
                  ('pca1', PCA(n_components=20)),
                  ('poly', PolynomialFeatures(degree=3)),
                  ('pca2', PCA(n_components=20)),
                  ('dtc', DecisionTreeClassifier(max_depth=3))])

In [14]:
model.fit(X_train, y_train)
model.score(X_test, y_test)

0.542

# Your model

In [16]:
# here